In [1]:
from __future__ import division
import json
from pprint import pprint
import pickle
import matplotlib.pyplot as plt
from plotLayer import *
from preprocess import *

import os
import time
import numpy as np
import glob
from scipy import sparse as sp
from skimage.measure import block_reduce
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, precision_recall_curve

from model import *
from keras.backend.tensorflow_backend import set_session

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


def Classify_Rate(y, y_hat):
    # True Positive (TP): we predict a label of 1 (positive), and the true label is 1.
    TP = np.sum(np.logical_and(y_hat == 1, y == 1))
    # True Negative (TN): we predict a label of 0 (negative), and the true label is 0.
    TN = np.sum(np.logical_and(y_hat == 0, y == 0))
    # False Positive (FP): we predict a label of 1 (positive), but the true label is 0.
    FP = np.sum(np.logical_and(y_hat == 1, y == 0))
    # False Negative (FN): we predict a label of 0 (negative), but the true label is 1.
    FN = np.sum(np.logical_and(y_hat == 0, y == 1))
    return TP, TN, FP, FN


def Predict(model, x, y_threshold=None):
    y_hat = model.predict(x)

    if y_threshold :
        y_hat[y_hat < y_threshold] = 0
        y_hat[y_hat >= y_threshold] = 1
    return y_hat

def PlotROC(y, y_hat):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(2):
        fpr[i], tpr[i], _ = roc_curve(y, y_hat)
        roc_auc[i] = auc(fpr[i], tpr[i])
    print ("roc_auc_score:%f" %roc_auc_score(y, y_hat))
    plt.figure()
    plt.plot(fpr[1], tpr[1])
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.show()

def PreprocessData(datapath, width=256, channel=6):
    ratio = 1024 // width
    with open(datapath, "rb") as f:
        data = pickle.load(f)
    x = np.heaviside(np.array([map(lambda x: block_reduce(x.toarray(), block_size=(ratio,ratio), func=np.max), d.hL) for d in data]), 0)
    x = np.swapaxes(x, 1, 3)
    y = np.array([d.label for d in data])
    return x, y


if __name__ == "__main__":
    width = 256
    channel = 6
    #classify_weights_path = "Classify_epoch_50_batch_4.hdf5"
    classify_weights_path = "Classify_epoch_100_batch_5.hdf5"


    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
    config.gpu_options.per_process_gpu_memory_fraction = 0.9
    sess = tf.Session(config=config)
    set_session(sess)

    classify_model = Encoder_Classify(input_size=(width,width,6), batch_normal=True)
    classify_model.load_weights(classify_weights_path)

    d0_path = "../Data/1stDataset/d0*"
    false_path = "../Data/1stDataset/false*"
    d0 = np.sort(glob.glob(d0_path))
    f0 = np.sort(glob.glob(false_path))

    y_total = []
    y_hat_total = []
#     301
    for i in range(250,301):
        x1, y1 = PreprocessData(d0[i], width=width, channel=channel)
        x2, y2 = PreprocessData(f0[i], width=width, channel=channel)
        x = np.concatenate((x1, x2), axis=0)
        y = np.concatenate((y1, y2), axis=0)

        y_hat = Predict(classify_model, x)
        y_total += [y]
        y_hat_total += [y_hat]
        print("y_hat mean:%f median:%f" %(np.mean(y_hat), np.median(y_hat)))
        y_hat[y_hat < np.median(y_hat)] = 0
        y_hat[y_hat >= np.median(y_hat)] = 1

        TP, TN, FP, FN = Classify_Rate(y, y_hat)
        print("Sensitive TP/(TP+FN) : %f" %(TP / (TP + FN)))
        print(classification_report(y, y_hat))

    y = np.concatenate(y_total)
    y_hat = np.concatenate(y_hat_total)
    print(y.shape)
    print(y_hat.shape)
    a=np.hstack((y,y_hat))
    np.savetxt('values_250_301.txt', a, delimiter =',')
    print("y_hat mean:%f median:%f" %(np.mean(y_hat), np.median(y_hat)))
    fp,tp,tr=roc_curve(y,y_hat)
    p,r,tr1=precision_recall_curve(y,y_hat)
    plt.plot(fp,tp,color='darkorange',lw=2,label='ROC curve(area = %0.3f)'%auc(fp,tp))
    plt.xlabel('False positive Rate')
    plt.ylabel('True positive Rate')
    legend = plt.legend(fontsize = 'x-large')
#     plt.savefig('ROC_curve.png')
    plt.show()
    
    
    plt.plot(r,p,color='darkorange', label='Precision recall curve') 
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    legend=plt.legend(fontsize='x-large')
#     plt.savefig('Precision_Recall.png')
    plt.show()
    y_hat[y_hat < np.median(y_hat)] = 0
    y_hat[y_hat >= np.median(y_hat)] = 1
    TP, TN, FP, FN = Classify_Rate(y, y_hat)
    print("Sensitive TP/(TP+FN) : %f" %(TP / (TP + FN)))
    print(classification_report(y, y_hat))
    



Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      3520      
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
__________

y_hat mean:0.346977 median:0.363431
Sensitive TP/(TP+FN) : 0.420000
              precision    recall  f1-score   support

           0       0.42      0.42      0.42       100
           1       0.42      0.42      0.42       100

   micro avg       0.42      0.42      0.42       200
   macro avg       0.42      0.42      0.42       200
weighted avg       0.42      0.42      0.42       200

y_hat mean:0.322503 median:0.307645
Sensitive TP/(TP+FN) : 0.390000
              precision    recall  f1-score   support

           0       0.39      0.39      0.39       100
           1       0.39      0.39      0.39       100

   micro avg       0.39      0.39      0.39       200
   macro avg       0.39      0.39      0.39       200
weighted avg       0.39      0.39      0.39       200

y_hat mean:0.344677 median:0.335938
Sensitive TP/(TP+FN) : 0.300000
              precision    recall  f1-score   support

           0       0.30      0.30      0.30       100
           1       0.30      0.30

y_hat mean:0.332918 median:0.322616
Sensitive TP/(TP+FN) : 0.420000
              precision    recall  f1-score   support

           0       0.42      0.42      0.42       100
           1       0.42      0.42      0.42       100

   micro avg       0.42      0.42      0.42       200
   macro avg       0.42      0.42      0.42       200
weighted avg       0.42      0.42      0.42       200

y_hat mean:0.349918 median:0.369728
Sensitive TP/(TP+FN) : 0.440000
              precision    recall  f1-score   support

           0       0.44      0.44      0.44       100
           1       0.44      0.44      0.44       100

   micro avg       0.44      0.44      0.44       200
   macro avg       0.44      0.44      0.44       200
weighted avg       0.44      0.44      0.44       200

y_hat mean:0.336657 median:0.334946
Sensitive TP/(TP+FN) : 0.390000
              precision    recall  f1-score   support

           0       0.39      0.39      0.39       100
           1       0.39      0.39

y_hat mean:0.351061 median:0.350379
Sensitive TP/(TP+FN) : 0.430000
              precision    recall  f1-score   support

           0       0.43      0.43      0.43       100
           1       0.43      0.43      0.43       100

   micro avg       0.43      0.43      0.43       200
   macro avg       0.43      0.43      0.43       200
weighted avg       0.43      0.43      0.43       200

y_hat mean:0.310189 median:0.276437
Sensitive TP/(TP+FN) : 0.370000
              precision    recall  f1-score   support

           0       0.37      0.37      0.37       100
           1       0.37      0.37      0.37       100

   micro avg       0.37      0.37      0.37       200
   macro avg       0.37      0.37      0.37       200
weighted avg       0.37      0.37      0.37       200

y_hat mean:0.323179 median:0.318096
Sensitive TP/(TP+FN) : 0.490000
              precision    recall  f1-score   support

           0       0.49      0.49      0.49       100
           1       0.49      0.49

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

Sensitive TP/(TP+FN) : 0.385023
              precision    recall  f1-score   support

           0       0.38      0.38      0.38      5099
           1       0.39      0.39      0.39      5101

   micro avg       0.39      0.39      0.38     10200
   macro avg       0.39      0.38      0.38     10200
weighted avg       0.39      0.39      0.39     10200



# ROC curve area = 0.36